In [15]:
import numpy as np
from datetime import datetime
import pandas as pd 
import time
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from decimal import Decimal

In [3]:
import seaborn as sns
import xgboost as xgb
from sklearn import metrics
from  sklearn.datasets  import  make_hastie_10_2
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import classification_report
from xgboost import plot_importance
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit  
from sklearn.preprocessing import StandardScaler    

In [7]:
nn = pd.read_excel("data_store_y_poss/nn.xlsx").iloc[:,1:]

In [4]:
lightgbm = pd.read_excel("data_store_y_poss/lightgbm.xlsx").iloc[:,1:]

In [8]:
cnn = np.load("data_store_y_poss/y_cnn.npy")
lstm = np.load("data_store_y_poss/y_lstm.npy")
resnet = np.load("data_store_y_poss/y_resnet.npy")

In [9]:
# make suer
(nn.iloc[:,:2] == lightgbm.iloc[:,:2])["date"].sum()

53089

In [10]:
df_base = nn.copy()
df_base

,date,code,uid,change_5_class6
0,2021-01-04,000001.XSHE,2021-01-04000001.XSHE,6
1,2021-01-04,000002.XSHE,2021-01-04000002.XSHE,6
2,2021-01-04,000012.XSHE,2021-01-04000012.XSHE,1
3,2021-01-04,000027.XSHE,2021-01-04000027.XSHE,2
4,2021-01-04,000039.XSHE,2021-01-04000039.XSHE,5
...,...,...,...,...
53084,2021-09-23,601998.XSHG,2021-09-23601998.XSHG,4
53085,2021-09-23,603019.XSHG,2021-09-23603019.XSHG,2
53086,2021-09-23,603288.XSHG,2021-09-23603288.XSHG,6
53087,2021-09-23,603806.XSHG,2021-09-23603806.XSHG,5


In [11]:
#get the class content
df_base["cnn"] = cnn.argmax(axis=1)
df_base["lstm"] = lstm.argmax(axis=1)
df_base["resnet"] = resnet.argmax(axis=1)
df_base["gbm"] = lightgbm["class_pre"]

In [37]:
df_base["ensemble"] = (0.5+(0.2*df_base["cnn"]+0.2*df_base["lstm"]+0.2*df_base["resnet"]+0.4*df_base["gbm"])/1)
df_base["ensemble"] = [int(i) for i in df_base["ensemble"]]
df_base

,date,code,uid,change_5_class6,cnn,lstm,resnet,gbm,ensemble
0,2021-01-04,000001.XSHE,2021-01-04000001.XSHE,6,5,5,5,5,5
1,2021-01-04,000002.XSHE,2021-01-04000002.XSHE,6,4,3,2,3,3
2,2021-01-04,000012.XSHE,2021-01-04000012.XSHE,1,2,2,5,2,3
3,2021-01-04,000027.XSHE,2021-01-04000027.XSHE,2,3,2,3,2,2
4,2021-01-04,000039.XSHE,2021-01-04000039.XSHE,5,2,2,5,2,3
...,...,...,...,...,...,...,...,...,...
53084,2021-09-23,601998.XSHG,2021-09-23601998.XSHG,4,3,2,4,4,3
53085,2021-09-23,603019.XSHG,2021-09-23603019.XSHG,2,4,3,3,2,3
53086,2021-09-23,603288.XSHG,2021-09-23603288.XSHG,6,5,3,2,2,3
53087,2021-09-23,603806.XSHG,2021-09-23603806.XSHG,5,6,5,5,1,4


In [38]:
# evaluate cnn
y_true = df_base["change_5_class6"]
y_pred = df_base["cnn"]
judge = classification_report(y_true, y_pred)
print(judge)

              precision    recall  f1-score   support

           1       0.24      0.06      0.10      4759
           2       0.24      0.38      0.29     11484
           3       0.28      0.17      0.21     10911
           4       0.27      0.44      0.33     10809
           5       0.19      0.16      0.17      9405
           6       0.19      0.05      0.07      5721

    accuracy                           0.25     53089
   macro avg       0.24      0.21      0.20     53089
weighted avg       0.24      0.25      0.22     53089



In [39]:
# evaluate lstm
y_true = df_base["change_5_class6"]
y_pred = df_base["lstm"]
judge = classification_report(y_true, y_pred)
print(judge)

              precision    recall  f1-score   support

           1       0.21      0.12      0.15      4759
           2       0.24      0.42      0.30     11484
           3       0.28      0.23      0.26     10911
           4       0.28      0.19      0.23     10809
           5       0.19      0.26      0.22      9405
           6       0.17      0.03      0.05      5721

    accuracy                           0.24     53089
   macro avg       0.23      0.21      0.20     53089
weighted avg       0.24      0.24      0.22     53089



In [40]:
# evaluate resnet
y_true = df_base["change_5_class6"]
y_pred = df_base["resnet"]
judge = classification_report(y_true, y_pred)
print(judge)

              precision    recall  f1-score   support

           1       0.25      0.04      0.07      4759
           2       0.23      0.31      0.26     11484
           3       0.27      0.18      0.22     10911
           4       0.27      0.39      0.32     10809
           5       0.18      0.25      0.21      9405
           6       0.16      0.04      0.07      5721

    accuracy                           0.23     53089
   macro avg       0.23      0.20      0.19     53089
weighted avg       0.23      0.23      0.22     53089



In [41]:
# evaluate ensemble
y_true = df_base["change_5_class6"]
y_pred = df_base["ensemble"]
judge = classification_report(y_true, y_pred)
print(judge)

              precision    recall  f1-score   support

           1       0.28      0.04      0.08      4759
           2       0.25      0.20      0.22     11484
           3       0.22      0.46      0.30     10911
           4       0.25      0.39      0.31     10809
           5       0.20      0.07      0.10      9405
           6       0.18      0.00      0.00      5721

    accuracy                           0.23     53089
   macro avg       0.23      0.19      0.17     53089
weighted avg       0.23      0.23      0.20     53089



In [44]:
date_list = df_base["date"].unique().tolist()
date_list_5 = []
date_list_5_buy = []
count = 0
for i in date_list:
    if (count % 5) == 0:
        date_list_5.append(i)
    if ((count-1) % 5) == 0:
        date_list_5_buy.append(i)
    count += 1

In [46]:
# change the tail
def stock_tail(series):
    if  series[-1:] == "E":
        return series[:-4]+"SZ"
    if  series[-1:] == "G":
        return series[:-4]+"SH"

In [47]:
df3 = df_base.copy()
df3 = df3[df3["date"].isin(date_list_5)]
df3 = df3[df3["ensemble"].isin([5,6])]
df3 = df3.reset_index(drop=True)
df3["new_code"] = df3.apply(lambda x:stock_tail(x["code"]),axis = 1)

In [53]:
import random
buy_dict = {}
for i in range(36):
    try:
        random_list = random.sample(df3[df3["date"] == date_list_5[i]]["new_code"].tolist(),10)
    except:
        buy_dict[date_list_5_buy[i]] =[]
    buy_dict[date_list_5_buy[i]] = random_list

In [55]:
print(buy_dict)

{'2021-01-05': ['601933.SH', '600085.SH'], '2021-01-12': ['000012.SZ', '600079.SH', '600754.SH', '600196.SH', '601216.SH', '000869.SZ', '300033.SZ', '600739.SH', '000423.SZ', '601336.SH'], '2021-01-19': ['601336.SH', '601021.SH', '002304.SZ', '000869.SZ', '600276.SH', '600066.SH', '000625.SZ', '600340.SH', '600600.SH', '601888.SH'], '2021-01-26': ['601216.SH', '002465.SZ', '601238.SH', '300253.SZ', '002456.SZ', '000651.SZ', '601099.SH', '000800.SZ', '600066.SH', '600271.SH'], '2021-02-02': ['002179.SZ', '002241.SZ', '600685.SH', '002049.SZ', '002065.SZ', '600418.SH', '002465.SZ', '002081.SZ', '600739.SH', '002456.SZ'], '2021-02-09': ['600741.SH', '002373.SZ', '000012.SZ', '002399.SZ', '600157.SH', '600588.SH', '002030.SZ', '600703.SH', '002475.SZ', '002223.SZ'], '2021-02-23': ['002410.SZ', '600038.SH', '000625.SZ', '600340.SH', '000333.SZ', '600741.SH', '600660.SH', '600893.SH', '002475.SZ', '600690.SH'], '2021-03-02': ['002444.SZ', '300124.SZ', '000895.SZ', '002475.SZ', '300168.SZ', '